# The Mapillary Traffic Sign Dataset for Detection and Classification on a Global Scale - paper review

## Challenges
- traffic signs are easily confused with other object classes in street
- reflection, low light condition, damages, and occlusion
- fine–grained classification
- traffic signs are relatively small in size

## Dataset Statistics
- images: 52,453 fully-anotated 47,547 partialy-anotated
- sign categories: 313 + 1 (other sign)
- total signs: 257 543

| train  | dev   | test   |
|--------|-------|--------|
| 36 589 | 5 320 | 10 544 |

- distribution plots are present in the paper

##  Annotation Process 
The annotations were done by 15 experts trained on this task. The authors continuously controlled the quality of annotations. At least two annotators must have seen each image. To further validate the quality of annotations, they runed separate annotation experiment over smaller subset of images and cross-checked the results showing only minor differences.

### 1. Selection
The images were selected using the following criteria:
- uniform geographical distribution of images around the world (weighted by continent population)
- to cover images of different quality, captured under varying conditions
- to include as many signs as possible per image
- to compensate for the long-tailed distribution of potential traffic
sign classes

### 2. Annotation
The annotation pipeline consisted of 3 steps:
1. Image Approval: the annotators should have ensured that the data fulfil the dataset criteria since the pre-selection was automatically
2. Sign Localization: The bounding boxes were pre-generated automatically. The annotators were asked to verify and adjust the bounding boxes to fit all traffic signs in the image.
3. Sign Classification: The annotators were asked to provide a correct class label for show sign (determined by box). This was not trivial since they used 313 classes. Thereby, the signs were pre-annotated automatically using a proposal network.

## Baseline
- Faster R-CNN with ResNet50 and ResNet101 back-bones
- two tasks: detection only and detection + classification
- ResNet50: 83.4 mAP over all 313 classes
- their best performing approach used 2 stage pipeline: 1. binary object detection, 2. multi-class classification using a decoupled shallow classification network


In [ ]:
import os
import numpy as np
import json
import shutil
from tqdm import tqdm
import matplotlib.pyplot as plt

In [ ]:
PATH = "C:\\Users\\tlust\\Downloads\\mtsd"

# input files
splits_path = os.path.join(PATH, "splits")
images_path = os.path.join(PATH, "images")
annotations_path = os.path.join(PATH, "annotations")

# output files
output_path = os.path.join(PATH, "yolov8")

# YOLOv8 parser

In [ ]:
labels = []

# statistics
rejected = 0
total = 0
sign_distr = {}

for split in ['train', 'val', 'test']:
    print("Processing {} split...".format(split))

    # 0. create output directories if not exists
    out_dir = os.path.join(output_path, split)
    if not os.path.exists(out_dir):
        os.makedirs(os.path.join(out_dir, "images"))
        os.makedirs(os.path.join(out_dir, "labels"))

    with open(os.path.join(splits_path, split + ".txt")) as f:
        ids = f.readlines()

    for id in tqdm(ids, total=len(ids)):
        total += 1    

        # 1. set and validate paths
        id = id.strip()
        img_path = os.path.join(images_path, f"{id}.jpg")
        ann_path = os.path.join(annotations_path, f"{id}.json")
        out_img_path = os.path.join(out_dir, "images", f"{id}.jpg")
        out_ann_path = os.path.join(out_dir, "labels", f"{id}.txt")  
  
        # 1.2. skip if image or annotation does not exists
        if (not os.path.exists(img_path)) or (not os.path.exists(ann_path)):
            rejected += 1
            continue

        # 2. copy the image
        shutil.copy(img_path, out_img_path)

        # 3. create YOLOv8 annotation
        with open(ann_path, 'r') as f:
            ann = json.load(f)
            
        with open(out_ann_path, "a") as f:  
            for obj in ann['objects']:
                # 3.1 get label index
                if obj['label'] not in labels:
                    labels.append(obj['label'])
                label = labels.index(obj['label'])

                # 3.2 set sign distribution
                sign_distr[label] = sign_distr[label] + 1 if label in sign_distr else 1

                # 3.3 get bounding box
                bbox = obj['bbox']
                x_center = ((bbox['xmin'] + bbox['xmax']) / 2) / ann['width']
                y_center = ((bbox['ymin'] + bbox['ymax'] ) / 2) / ann['height']
                width = (bbox['xmax'] - bbox['xmin']) / ann['width']
                height = (bbox['ymax'] - bbox['ymin']) / ann['height']
                obj_ann = f"{label} {x_center} {y_center} {width} {height} \n"

                # 3.4 write annotation
                f.write(obj_ann)



In [ ]:
print("Images - total: {}".format(total))
print("Images - rejected: {}".format(rejected) + " ({:.2f}%)".format(rejected / total * 100))
print("Signs: {}".format(np.sum(sign_distr.values())))
plt.bar(labels, sign_distr)